## Notebook to generate collision rate violators for any RMG job while running

In [ ]:
import sys, os
sys.path.append('/Users/ebubeajulu/anaconda3/envs/rmg_py3/')

!conda list rmg_py3
!which python

In [14]:
import logging

In [ ]:
import rmgpy

import numpy as np

from rmgpy.rmgobject import RMGObject, expand_to_dict, recursive_make_object


In [5]:
%cat chemkin/chem.inp

ELEMENTS
	H
	D /2.014/
	T /3.016/
	C
	CI /13.003/
	O
	OI /18.000/
	N
	Ne
	Ar
	He
	Si
	S
	Cl
	X /195.083/
END

SPECIES
    N2              
    Ar              
    Ne              
    nC10H22(1)      
    O2(2)           
    CH(3)           
    C2H(4)          
    O(5)            
    CO(6)           
    CO2(7)          
    H2O(8)          
    CH2O(9)         
    CH4(10)         
    C2H4(11)        
    C3H6(12)        
    H2(13)          
    H(14)           
    OH(15)          
    He(16)          
    HO2(17)         
    H2O2(18)        
    HOCO(19)        
    HCO(20)         
    CH3(21)         
    CH2(22)         
    CH3OH(23)       
    CH2(S)(24)      
    CH2OH(25)       
    CH3O(26)        
    HCOH(27)        
    CH3OO(28)       
    CH2CO(29)       
    C2H5(30)        
    C2H3(31)        
    C(32)           
    C2H2(33)        
    CH3OOH(34)      
    CH2OOH(35)      
    C2H6(36)        
    CH3CHO(37)      
    C2H5O(38)       
    C2H5O2(39)      


CH2(S)(24)+C9H19(690)=S(692)                        7.188190e+10 0.444     0.000    
DUPLICATE

CH2(22)+C9H19(690)=S(692)                           1.067320e+12 0.473     0.000    

C9H19(690)+S(691)=S(1547)+nC10H22(1)                1.026000e+14 -0.350    0.000    

C9H19(690)+S(686)=S(1547)+nC10H22(1)                1.026000e+14 -0.350    0.000    

C9H19(690)+S(688)=S(1547)+nC10H22(1)                1.026000e+14 -0.350    0.000    

C9H19(690)+S(689)=S(1547)+nC10H22(1)                1.026000e+14 -0.350    0.000    

C9H19(690)+S(692)=S(1547)+nC10H22(1)                2.900000e+12 0.000     0.000    

O2(2)+C9H19(690)=HO2(17)+S(1547)                    1.833000e+13 0.000     14.850   

HO2(17)+C9H19(690)=H2O2(18)+S(1547)                 4.820000e+13 0.000     0.000    

CH2OH(25)+C9H19(690)=CH3OH(23)+S(1547)              9.640000e+11 0.000     0.000    

CH3O(26)+C9H19(690)=CH3OH(23)+S(1547)               4.820000e+13 0.000     0.000    

CH2OOH(35)+C9H19(690)=CH3OOH(34)+S(1547)    

In [25]:
import rmgpy 
from rmgpy.chemkin import load_chemkin_file
species, reactions = rmgpy.chemkin.load_chemkin_file('chemkin/chem.inp','chemkin/species_dictionary.txt',
                                                     check_duplicates=False,
                                                    use_chemkin_names=True,
)


In [31]:
species_dict = {rmgpy.chemkin.get_species_identifier(s): s for s in species}
species_dict.keys()

dict_keys(['N2', 'Ar', 'Ne', 'nC10H22(1)', 'O2(2)', 'CH(3)', 'C2H(4)', 'O(5)', 'CO(6)', 'CO2(7)', 'H2O(8)', 'CH2O(9)', 'CH4(10)', 'C2H4(11)', 'C3H6(12)', 'H2(13)', 'H(14)', 'OH(15)', 'He(16)', 'HO2(17)', 'H2O2(18)', 'HOCO(19)', 'HCO(20)', 'CH3(21)', 'CH2(22)', 'CH3OH(23)', 'CH2(S)(24)', 'CH2OH(25)', 'CH3O(26)', 'HCOH(27)', 'CH3OO(28)', 'CH2CO(29)', 'C2H5(30)', 'C2H3(31)', 'C(32)', 'C2H2(33)', 'CH3OOH(34)', 'CH2OOH(35)', 'C2H6(36)', 'CH3CHO(37)', 'C2H5O(38)', 'C2H5O2(39)', 'CH2CHO(40)', 'H2CC(41)', 'C2H4O(42)', 'C2H5O(43)', 'cC2H4O(44)', 'C2H5O2(45)', 'C2H3O2(46)', 'CHCHO(47)', 'OCHCHO(48)', 'HCCO(49)', 'HCCOH(50)', 'CHCHOH(51)', 'C2(52)', 'C2O(53)', 'C2H6O(54)', 'C2H5O(55)', 'C2H5O3(56)', 'CH3CO(57)', 'cC2H3O(58)', 'HOCHO(59)', 'C2H3O3(60)', 'OCHCO(61)', 'C2H6O2(62)', 'C2H5O2(63)', 'C2H4O2(64)', 'HOCH2O(65)', 'OCHO(66)', 'C2H4O3(67)', 'C2H3O2(68)', 'NC3H7(101)', 'C3H5(103)', 'S(104)', 'S(123)', 'C3H4(174)', 'PC4H9(190)', 'S(192)', 'C4H8(196)', 'S(227)', 'S(228)', 'C4H6O(271)', 'C5H11(4

In [32]:

rmgpy.chemkin.load_transport_file('chemkin/tran.dat', species_dict)


In [35]:
def check_collision_limit_violation(self, t_min, t_max, p_min, p_max):
        """
        Warn if a core reaction violates the collision limit rate in either the forward or reverse direction
        at the relevant extreme T/P conditions. Assuming a monotonic behaviour of the kinetics.
        Returns a list with the reaction object and the direction in which the violation was detected.
        """
        conditions = [[t_min, p_min]]
        if t_min != t_max:
            conditions.append([t_max, p_min])
        if self.kinetics.is_pressure_dependent() and p_max != p_min:
            conditions.append([t_min, p_max])
            if t_min != t_max:
                conditions.append([t_max, p_max])
        logging.debug("Checking whether reaction {0} violates the collision rate limit...".format(self))
        violator_list = []
        kf_list = []
        kr_list = []
        collision_limit_f = []
        collision_limit_r = []
        for condition in conditions:
            if len(self.reactants) >= 2:
                try:
                    collision_limit_f.append(self.calculate_coll_limit(temp=condition[0], reverse=False))
                except ValueError:
                    continue
                else:
                    kf_list.append(self.get_rate_coefficient(condition[0], condition[1]))
            if len(self.products) >= 2:
                try:
                    collision_limit_r.append(self.calculate_coll_limit(temp=condition[0], reverse=True))
                except ValueError:
                    continue
                else:
                    kr_list.append(self.generate_reverse_rate_coefficient().get_rate_coefficient(condition[0], condition[1]))
        if len(self.reactants) >= 2:
            for i, k in enumerate(kf_list):
                if k > collision_limit_f[i]:
                    ratio = k / collision_limit_f[i]
                    condition = '{0} K, {1:.1f} bar'.format(conditions[i][0], conditions[i][1] / 1e5)
                    violator_list.append([self, 'forward', ratio, condition])
        if len(self.products) >= 2:
            for i, k in enumerate(kr_list):
                if k > collision_limit_r[i]:
                    ratio = k / collision_limit_r[i]
                    condition = '{0} K, {1:.1f} bar'.format(conditions[i][0], conditions[i][1] / 1e5)
                    violator_list.append([self, 'reverse', ratio, condition])
        return violator_list

In [41]:
import numpy as np
np.linalg.LinAlgError

numpy.linalg.linalg.LinAlgError

In [43]:
violators = []
for r in reactions:
    try:
        v = check_collision_limit_violation(r, 800,1000, 1e5, 30e5)
    except np.linalg.LinAlgError:
        print(f"LinAlgError checking {r}")
    if v:
        print(r)
        violators.extend(v)

H(14) + HO2(17) <=> H2O2(18)
LinAlgError checking O2(2) + CH2O(9) <=> HO2(17) + HCO(20)
LinAlgError checking HO2(17) + CH2O(9) <=> H2O2(18) + HCO(20)
LinAlgError checking CH2O(9) + CH3(21) <=> HCO(20) + CH4(10)
CH3(21) <=> H2(13) + CH(3)
CH3(21) <=> H(14) + CH2(22)


/Users/ebubeajulu/anaconda3/envs/rmg3/lib/python3.7/site-packages/ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in log


CH2(22) <=> H(14) + CH(3)
LinAlgError checking OH(15) + CH(3) <=> H(14) + HCO(20)
LinAlgError checking O2(2) + CH(3) <=> O(5) + HCO(20)
LinAlgError checking CO2(7) + CH(3) <=> CO(6) + HCO(20)
LinAlgError checking HCO(20) + CH3OH(23) <=> CH2O(9) + CH2OH(25)
LinAlgError checking HCO(20) + CH3OH(23) <=> CH2O(9) + CH3O(26)
LinAlgError checking O(5) + C2H4(11) <=> HCO(20) + CH3(21)
LinAlgError checking O(5) + C2H4(11) <=> HCO(20) + CH3(21)
LinAlgError checking CH2O(9) + C2H3(31) <=> HCO(20) + C2H4(11)
C2H2(33) <=> H(14) + C2H(4)
LinAlgError checking O(5) + C2H4O(42) <=> HCO(20) + CH2OH(25)
LinAlgError checking O(5) + C2H4O(42) <=> HCO(20) + CH2OH(25)
LinAlgError checking O2(2) + C2H4O(42) => OH(15) + HCO(20) + CH2O(9)
LinAlgError checking H(14) + CH3CO(57) <=> HCO(20) + CH3(21)
LinAlgError checking H(14) + CH2CHO(40) <=> HCO(20) + CH3(21)
LinAlgError checking O(5) + CH2CHO(40) <=> HCO(20) + CH2O(9)
LinAlgError checking OH(15) + CH2CHO(40) <=> HCO(20) + CH2OH(25)
LinAlgError checking CH2(22)

In [46]:
from rmgpy.data.kinetics import TemplateReaction, LibraryReaction
from rmgpy.rmg.pdep import PDepReaction

In [50]:
for violator in violators:
    rxn_string = str(violator[0])
    kinetics = violator[0].kinetics
    comment = ''
    if isinstance(violator[0], TemplateReaction):
        comment = violator[0].kinetics.comment
        violator[0].kinetics.comment = ''  # the comment is printed better when outside of the object
    if isinstance(violator[0], LibraryReaction):
        comment = 'Kinetic library: {0}'.format(violator[0].library)
    if isinstance(violator[0], PDepReaction):
        comment = 'Network #{0}'.format(violator[0].network)
    direction = violator[1]
    ratio = violator[2]
    condition = violator[3]
    
    if ratio < 5:
        continue # only print the big ones!
        
    print('{0}\n{1}\n{2}\nDirection: {3}\nViolation factor: {4:.2f}\n'
                      'Violation condition: {5}\n\n'.format(rxn_string, kinetics, comment, direction,
                                                            ratio, condition))
    if isinstance(violator[0], TemplateReaction):
        # although this is the end of the run, restore the original comment
        violator[0].kinetics.comment = comment

CH3(21) <=> H2(13) + CH(3)
ThirdBody(arrheniusLow=Arrhenius(A=(3.1e+15,'cm^3/(mol*s)'), n=0, Ea=(80.871,'kcal/mol'), T0=(1,'K')), efficiencies={})
Kinetic library: Unclassified
Direction: reverse
Violation factor: 48.56
Violation condition: 800 K, 1.0 bar


CH3(21) <=> H2(13) + CH(3)
ThirdBody(arrheniusLow=Arrhenius(A=(3.1e+15,'cm^3/(mol*s)'), n=0, Ea=(80.871,'kcal/mol'), T0=(1,'K')), efficiencies={})
Kinetic library: Unclassified
Direction: reverse
Violation factor: 1456.83
Violation condition: 800 K, 30.0 bar


CH3(21) <=> H2(13) + CH(3)
ThirdBody(arrheniusLow=Arrhenius(A=(3.1e+15,'cm^3/(mol*s)'), n=0, Ea=(80.871,'kcal/mol'), T0=(1,'K')), efficiencies={})
Kinetic library: Unclassified
Direction: reverse
Violation factor: 39.41
Violation condition: 1000 K, 30.0 bar


CH3(21) <=> H(14) + CH2(22)
ThirdBody(arrheniusLow=Arrhenius(A=(2.2e+15,'cm^3/(mol*s)'), n=0, Ea=(82.659,'kcal/mol'), T0=(1,'K')), efficiencies={})
Kinetic library: Unclassified
Direction: reverse
Violation factor: 76.45
